In [217]:
"""    
    Indicador: Rendimientos del Equipo de Reconocimiento para pago
"""

'    \n    Indicador: Rendimientos del Equipo de Reconocimiento para pago\n'

<h3>Importación Librerías</h3>

In [218]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime import datetime

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_dataframe_terrenos
import funcion_SQL_a_DataFrame

<h3>Parametrización de Variables</h3>

In [219]:
consulta_tRendimientosActualizacion = """
    select t2.radicado 
        ,t2.usuario_responsable_actividad_anterior
        ,t2.actividad_actual_tramite
        ,t2.usuario_actual_responsable_tramite
        ,t2.con_plano_cartografico
        ,t2.usuario_inserta_plano
        ,t2.npn
        ,t2.municipio
        ,(case when t2.actividad_actual_tramite in ('Consolidando información'
                                                    ,'Incorporando predios a la versión'
                                                    ,'Realizando visita de campo'
                                                    ,'Revisando Información Reconocimiento  Inicial'
                                                    ,'Revisando Información') then 'Reconocimiento'
            when t2.actividad_actual_tramite in ('Realizando Análisis Catastro Registro') then 'Jurídica'
            when t2.actividad_actual_tramite in ('Preparando Edición Geográfica') then 'SIG'
            else 'Sin parametrización'
            end) componente_responsable
        ,(case when actividad_actual_tramite in ('Consolidando información'
                                                ,'Incorporando predios a la versión'
                                                ,'Realizando visita de campo'
                                                ,'Revisando Información Reconocimiento  Inicial'
                                                ,'Realizando Análisis Catastro Registro'
                                                ,'Preparando Edición Geográfica') then 'Fase Previa a la Transacción Catastral'
                when actividad_actual_tramite in ('Revisando Información') then 'En Transacción Catastral o Posterior'
            else 'Sin parametrización'
            end) fase_actividad
            
    from
    (
    select t1.radicado
    ,t1.user_fn_activiad_ant usuario_responsable_actividad_anterior
    ,t1.actividad actividad_actual_tramite
    ,t1.user_actividad_act usuario_actual_responsable_tramite
    ,t1.plano_cartografico con_plano_cartografico
    ,t1.user_inserta_plano usuario_inserta_plano
    ,t1.npn 
    ,t1.municipio 
    from bpmcat.vw_trmactivos_rad_npn_plano t1
    where nom_tramite = 'Formación y Actualización'
    ) t2;
"""

orden_columnas_indicador_2 = ['Incorporando predios a la versión'
    ,'Realizando visita de campo'
    ,'Consolidando información'
    ,'Revisando Información Reconocimiento  Inicial'
]

columnas_croquis = ['municipio'
                    ,'radicado'
                    ,'componente_responsable'
                    ,'npn'
                    ,'usuario_actual_responsable_tramite']

dict_coordinador_reconocedor = {'abel.maldonado':'Coordinador',
                                'angel.banquez':'Reconocedor',
                                'celci.quintana':'Reconocedor',
                                'emilio.castillo':'Reconocedor',
                                'fabian.arango':'Reconocedor',
                                'fabio.barragan':'Reconocedor',
                                'german.panizza':'Reconocedor',
                                'harlingto.padilla':'Reconocedor',
                                'hernando.sarmiento':'Reconocedor',
                                'jacqueline.jimenez':'Coordinador',
                                'jarod.macareno':'Reconocedor',
                                'karen.escobar':'Reconocedor',
                                'luis.jimenez':'Reconocedor',
                                'mary.conquett':'Reconocedor',
                                'ricardo.guerrero':'Reconocedor',
                                'samith.saenz':'Reconocedor',
                                'william.perez':'Reconocedor',
                                'yesmy.batista':'Coordinador',
                                'andres.ramos':'Editor SIG',
                                'melany.delahoz':'Jurídico',
                                'kevin.castro':'Coordinador',
                                'stefany.dominguez':'Reconocedor',
                                'rafael.molina':'Reconocedor',
                                'jorge.hernandez':'Reconocedor',
                                'alvaro.donado':'Reconocedor',
                                'yeinez.cortes':'Reconocedor',
                                'pedro.garcia':'Reconocedor',
                                'nadith.miranda':'Reconocedor',
                                'juan.herreraa':'Sin información',
                                'maria.hernandez':'Reconocedor',
                                'sandra.acosta':'Sin información',
                                'julie.castano':'Jurídico',
                                'francisco.galarcio':'Reconocedor',
                                'monica.penuela':'Reconocedor',
                                'luis.jimenez':'Reconocedor',
                                'harlingto.padilla':'Reconocedor',
                                'euclides.polo':'Reconocedor',
                                'william.perez':'Reconocedor',}

dict_renombrado_columnas = {'radicado_y':'radicado',
                            'municipio_y':'municipio',
                            'usuario_responsable_actividad_anterior_y':'usuario_responsable_actividad_anterior',
                            'rol_anterior_responsable_actividad_y':'rol_anterior_responsable_actividad'
                            }

patron_busqueda = 'ransacci'

fecha_actual = datetime.now()
fecha_directorio = str(fecha_actual.strftime("%Y%m%d"))

DIRECTORIO_SALIDA_XLSX = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_2_Rendimientos_Reconocimiento"
NOMBRE_XLSX = str(fecha_directorio) + "_RendimientoEquipoReconocimiento_BCGS.xlsx"
RUTA_SALIDA_XLSX = os.path.join(DIRECTORIO_SALIDA_XLSX, NOMBRE_XLSX)

DIRECTORIO_EXCEL_BASE = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\zInsumos"
NOMBRE_EXCEL_BASE = 'base_npn_analisis_seguimiento.xlsx'
RUTA_EXCEL_BASE = os.path.join(DIRECTORIO_EXCEL_BASE, NOMBRE_EXCEL_BASE)

<h3>Ejecución de Funciones</h3>

In [220]:
df_consulta_rendimientosActualizacion = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_tRendimientosActualizacion)

Inicio de la conexión
Conexión exitosa ...


<h3>Incorporación de Rol</h3>

In [221]:
df_consulta_rendimientosActualizacion['rol_actual_responsable_tramite'] = None
df_consulta_rendimientosActualizacion['rol_anterior_responsable_actividad'] = None
for llave, valor in dict_coordinador_reconocedor.items():
    df_consulta_rendimientosActualizacion.loc[df_consulta_rendimientosActualizacion['usuario_actual_responsable_tramite']==llave, 'rol_actual_responsable_tramite'] = valor
    df_consulta_rendimientosActualizacion.loc[df_consulta_rendimientosActualizacion['usuario_responsable_actividad_anterior']==llave, 'rol_anterior_responsable_actividad'] = valor

<h3>1. Análisis Rendimientos sobre Reconocimiento</h3>
<ol>
    <li> Indicador 1: Total de Predios por Fase de Actividad y Reconocedor (Eliminar)
    <li> Indicador 2: Total de predios, fase previa a Transacción, por usuario
    <li> Indicador 3: Total de predios, fase Transacción o posterior, por usuario
    <li> Indicador 4: Total de predios, que fueron liberados por reconocimiento previo análisis SIG
</ol>

In [222]:

# # ? Indicador 1
# df_rendActua_ERecono = df_consulta_rendimientosActualizacion[df_consulta_rendimientosActualizacion['componente_responsable']=='Reconocimiento']
# df_rendActua_ERecono_indicador_1 = pd.DataFrame(df_rendActua_ERecono.groupby(['municipio','fase_actividad','rol_actual_responsable_tramite'])['usuario_actual_responsable_tramite'].value_counts()).reset_index().rename(columns={'count':'total_predios'})
# df_rendActua_ERecono_indicador_1['rol_usuario'] = df_rendActua_ERecono_indicador_1['rol_actual_responsable_tramite'] + ' - ' + df_rendActua_ERecono_indicador_1['usuario_actual_responsable_tramite']
# print(f"Indicador 1: \n {df_rendActua_ERecono_indicador_1}")

# ? Indicador 2
df_rendActua_ERecono_FasePrevia = df_consulta_rendimientosActualizacion[(df_consulta_rendimientosActualizacion['componente_responsable']=='Reconocimiento') & (df_consulta_rendimientosActualizacion['fase_actividad']=='Fase Previa a la Transacción Catastral')]

# TODO: Se crea diccionario para ordenar columnas de acuerdo a flujo de trabajo de Actualización
dict_orden_columnas_ind_2 = {valor: index for index, valor in enumerate(orden_columnas_indicador_2)}

df_rendActua_ERecono_indicador_2 = pd.DataFrame(df_rendActua_ERecono_FasePrevia.groupby(['municipio','actividad_actual_tramite','rol_actual_responsable_tramite'])['usuario_actual_responsable_tramite'].value_counts()).reset_index().rename(columns={'count':'total_predios'})

# TODO: Ordenar el DataFrame usando el diccionario de mapeo
df_rendActua_ERecono_indicador_2['actividad_ordenada_s_flujo'] = df_rendActua_ERecono_indicador_2['actividad_actual_tramite'].map(dict_orden_columnas_ind_2)
df_rendActua_ERecono_indicador_2_Ordenado = df_rendActua_ERecono_indicador_2.sort_values(by='actividad_ordenada_s_flujo').drop(columns='actividad_ordenada_s_flujo')

# ** Filtro Municipio
# ! María La Baja
df_rendActua_ERecono_indicador_2_Ordenado_Maria = df_rendActua_ERecono_indicador_2_Ordenado[df_rendActua_ERecono_indicador_2_Ordenado['municipio']=='MARÍA LA BAJA']
df_rendActua_ERecono_indicador_2_Ordenado_Repelon = df_rendActua_ERecono_indicador_2_Ordenado[df_rendActua_ERecono_indicador_2_Ordenado['municipio']=='REPELÓN']

# ! Copia
df_rendActua_ERecono_indicador_2_Ordenado_Maria = df_rendActua_ERecono_indicador_2_Ordenado_Maria.copy()
df_rendActua_ERecono_indicador_2_Ordenado_Repelon = df_rendActua_ERecono_indicador_2_Ordenado_Repelon.copy()

# ! Unificacion Campos
df_rendActua_ERecono_indicador_2_Ordenado_Maria['rol_usuario'] = df_rendActua_ERecono_indicador_2_Ordenado_Maria['rol_actual_responsable_tramite'] + ' - ' + df_rendActua_ERecono_indicador_2_Ordenado_Maria['usuario_actual_responsable_tramite']
df_rendActua_ERecono_indicador_2_Ordenado_Repelon['rol_usuario'] = df_rendActua_ERecono_indicador_2_Ordenado_Repelon['rol_actual_responsable_tramite'] + ' - ' + df_rendActua_ERecono_indicador_2_Ordenado_Repelon['usuario_actual_responsable_tramite']

# ! Pivote
df_rendActua_ERecono_indicador_2_Ordenado_Maria_pivote = df_rendActua_ERecono_indicador_2_Ordenado_Maria.pivot(index='rol_usuario', columns='actividad_actual_tramite', values='total_predios')
df_rendActua_ERecono_indicador_2_Ordenado_Repelon_pivote = df_rendActua_ERecono_indicador_2_Ordenado_Repelon.pivot(index='rol_usuario', columns='actividad_actual_tramite', values='total_predios')

# ! Eliminación de nulos
for columna in df_rendActua_ERecono_indicador_2_Ordenado_Maria_pivote.columns:
    df_rendActua_ERecono_indicador_2_Ordenado_Maria_pivote.loc[df_rendActua_ERecono_indicador_2_Ordenado_Maria_pivote[columna].isnull(), columna] = 0
    df_rendActua_ERecono_indicador_2_Ordenado_Maria_pivote[columna] = df_rendActua_ERecono_indicador_2_Ordenado_Maria_pivote[columna].astype(int)

for columna in df_rendActua_ERecono_indicador_2_Ordenado_Repelon_pivote.columns:
    df_rendActua_ERecono_indicador_2_Ordenado_Repelon_pivote.loc[df_rendActua_ERecono_indicador_2_Ordenado_Repelon_pivote[columna].isnull(), columna] = 0
    df_rendActua_ERecono_indicador_2_Ordenado_Repelon_pivote[columna] = df_rendActua_ERecono_indicador_2_Ordenado_Repelon_pivote[columna].astype(int)

# ! Reseteo

df_rendActua_ERecono_indicador_2_Ordenado_Maria_pivote = df_rendActua_ERecono_indicador_2_Ordenado_Maria_pivote.reset_index()
df_rendActua_ERecono_indicador_2_Ordenado_Repelon_pivote = df_rendActua_ERecono_indicador_2_Ordenado_Repelon_pivote.reset_index()

# # ? Indicador 3
# df_rendActua_ERecono_FaseTransaccionOPost = df_consulta_rendimientosActualizacion[(df_consulta_rendimientosActualizacion['componente_responsable']=='Reconocimiento') & 
#                                                                                   (df_consulta_rendimientosActualizacion['fase_actividad']=='En Transacción Catastral o Posterior')]

# df_rendActua_ERecono_indicador_3 = pd.DataFrame(df_rendActua_ERecono_FaseTransaccionOPost.groupby(['municipio','actividad_actual_tramite','rol_actual_responsable_tramite'])['usuario_responsable_actividad_anterior'].value_counts()).reset_index().rename(columns={'count':'total_predios'})
# print(f"Indicador 3 \n {df_rendActua_ERecono_indicador_3}")

<h3>1. Análisis Rendimientos sobre Reconocimiento</h3>
<ol>
    <li> Indicador 4: Total de predios, que fueron liberados por reconocimiento previo análisis SIG
</ol>

In [223]:

# ? Indicador 4
# TODO: Creación archivo base
# df_PrediosPreparandoEdicionGeo = df_consulta_rendimientosActualizacion[(df_consulta_rendimientosActualizacion['actividad_actual_tramite'] == 'Preparando Edición Geográfica') |
#                                                                        (df_consulta_rendimientosActualizacion['actividad_actual_tramite'].str.contains(patron_busqueda))]
# df_PrediosPreparandoEdicionGeo_Base = df_PrediosPreparandoEdicionGeo[['radicado', 'municipio', 'rol_anterior_responsable_actividad', 'usuario_responsable_actividad_anterior', 'npn']]
# df_PrediosPreparandoEdicionGeo_Base = df_PrediosPreparandoEdicionGeo_Base.copy()
# df_PrediosPreparandoEdicionGeo_Base['fecha_registro'] = fecha_directorio
# df_PrediosPreparandoEdicionGeo_Base.to_excel(RUTA_EXCEL_BASE)

# TODO: Actualización del Archivo Base, partiendo del reporte XLSX creado el 17 de Junio, 2024
df_lectura_xlsx_base = pd.read_excel(RUTA_EXCEL_BASE)
df_lectura_xlsx_base = df_lectura_xlsx_base[['radicado','municipio','rol_anterior_responsable_actividad','usuario_responsable_actividad_anterior','npn','fecha_registro']]
print(f"Total de registros en archivo base: \n {df_lectura_xlsx_base.shape[0]}")

#! Edicióm Cartográfica
df_PrediosPreparandoEdicionGeo_Seguimiento = df_consulta_rendimientosActualizacion[(df_consulta_rendimientosActualizacion['actividad_actual_tramite'] == 'Preparando Edición Geográfica') |
                                                                                   (df_consulta_rendimientosActualizacion['actividad_actual_tramite'].str.contains(patron_busqueda))]
print(f"Total de registros seguimiento diario: \n {df_PrediosPreparandoEdicionGeo_Seguimiento.shape[0]}")

df_PrediosPreparandoEdicionGeo_Seguimiento = df_PrediosPreparandoEdicionGeo_Seguimiento[['radicado', 'municipio', 'rol_anterior_responsable_actividad', 'usuario_responsable_actividad_anterior', 'npn']]
df_cambios_base_seguimiento = pd.merge(df_lectura_xlsx_base, df_PrediosPreparandoEdicionGeo_Seguimiento, how = "right", on = "npn")
df_cambios_base_seguimiento_nuevos = df_cambios_base_seguimiento[df_cambios_base_seguimiento['radicado_x'].isnull()]
print(f"{df_cambios_base_seguimiento_nuevos} con {df_cambios_base_seguimiento_nuevos.shape[0]} nuevos registros")
if df_cambios_base_seguimiento_nuevos.shape[0] > 0:
    df_cambios_base_seguimiento_nuevos = df_cambios_base_seguimiento_nuevos[['radicado_y','municipio_y', 'rol_anterior_responsable_actividad_y', 'usuario_responsable_actividad_anterior_y','npn']]
    df_cambios_base_seguimiento_nuevos = df_cambios_base_seguimiento_nuevos.rename(columns=dict_renombrado_columnas)
    df_cambios_base_seguimiento_nuevos['fecha_registro'] = fecha_directorio
    df_cambios_base_seguimiento_unificacion = pd.concat([df_lectura_xlsx_base, df_cambios_base_seguimiento_nuevos])
    df_cambios_base_seguimiento_unificacion.to_excel(RUTA_EXCEL_BASE)
else:
    print(f"El archivo base no reporta nuevos predios en Edición Geográfica")

Total de registros en archivo base: 
 677
Total de registros seguimiento diario: 
 677
Empty DataFrame
Columns: [radicado_x, municipio_x, rol_anterior_responsable_actividad_x, usuario_responsable_actividad_anterior_x, npn, fecha_registro, radicado_y, municipio_y, rol_anterior_responsable_actividad_y, usuario_responsable_actividad_anterior_y]
Index: [] con 0 nuevos registros
El archivo base no reporta nuevos predios en Edición Geográfica


In [224]:
df_lectura_xlsx_base['rol_usuario'] = df_lectura_xlsx_base['rol_anterior_responsable_actividad'] + ' - ' + df_lectura_xlsx_base['usuario_responsable_actividad_anterior']
df_indicador_4 = pd.DataFrame(df_lectura_xlsx_base.groupby(['municipio','rol_usuario'])['fecha_registro'].value_counts()).reset_index().rename(columns={'count':'total_predios'})

df_indicador_4_maria = df_indicador_4[df_indicador_4['municipio'] == 'MARÍA LA BAJA']
df_indicador_4_maria_std = df_indicador_4_maria[['rol_usuario','fecha_registro','total_predios']]
df_indicador_4_maria_std_pivote = df_indicador_4_maria_std.pivot(index="rol_usuario", columns="fecha_registro", values="total_predios").reset_index()
df_indicador_4_maria_std_pivote


fecha_registro,rol_usuario,20240617
0,Reconocedor - angel.banquez,76
1,Reconocedor - emilio.castillo,44
2,Reconocedor - hernando.sarmiento,11
3,Reconocedor - maria.hernandez,55
4,Reconocedor - mary.conquett,121
5,Reconocedor - nadith.miranda,65
6,Reconocedor - rafael.molina,244
7,Sin información - sandra.acosta,61


<h3>Indicadores 1, 2 y 3 para Tabla XLSX<h3>

In [225]:

# # ? Indicador 1

# df_rendActua_ERecono_indicador_1_pivote = df_rendActua_ERecono_indicador_1.pivot(index='rol_usuario', columns='fase_actividad', values='total_predios')

# for columna in df_rendActua_ERecono_indicador_1_pivote.columns:
#     df_rendActua_ERecono_indicador_1_pivote.loc[df_rendActua_ERecono_indicador_1_pivote[columna].isnull(), columna] = 0
#     df_rendActua_ERecono_indicador_1_pivote[columna] = df_rendActua_ERecono_indicador_1_pivote[columna].astype(int)

# df_rendActua_ERecono_indicador_1_pivote = pd.DataFrame(df_rendActua_ERecono_indicador_1_pivote).reset_index()
# df_rendActua_ERecono_indicador_1_pivote


# # ? Indicador 3
# df_rendActua_ERecono_indicador_3_pivote = df_rendActua_ERecono_indicador_3.pivot(index='usuario_responsable_actividad_anterior', columns='actividad_actual_tramite', values='total_predios')

# for columna in df_rendActua_ERecono_indicador_3_pivote.columns:
#     df_rendActua_ERecono_indicador_3_pivote.loc[df_rendActua_ERecono_indicador_3_pivote[columna].isnull(), columna] = 0
#     df_rendActua_ERecono_indicador_3_pivote[columna] = df_rendActua_ERecono_indicador_3_pivote[columna].astype(int)

# df_rendActua_ERecono_indicador_3_pivote = pd.DataFrame(df_rendActua_ERecono_indicador_3_pivote).reset_index()


<h3>Reporte Croquis Vs NPN</h3>

In [226]:
df_npn_croquis = df_consulta_rendimientosActualizacion[df_consulta_rendimientosActualizacion['con_plano_cartografico']=='Si']
df_npn_croquis_estandarizado = df_npn_croquis[columnas_croquis]

<h3>Reporte NPN por Radicado</h3>

In [227]:
df_radicado_npn = df_consulta_rendimientosActualizacion[['municipio','componente_responsable','radicado','npn', 'actividad_actual_tramite','usuario_actual_responsable_tramite','con_plano_cartografico']]


<h3>Generación XLSX</h3>

In [228]:

# ** Usar ExcelWriter para guardar múltiples dataframes en diferentes hojas
with pd.ExcelWriter(RUTA_SALIDA_XLSX, engine='xlsxwriter') as writer:
    # df_rendActua_ERecono_indicador_1_pivote.to_excel(writer, sheet_name='Indicador_1', index=False)
    df_rendActua_ERecono_indicador_2_Ordenado_Maria_pivote.to_excel(writer, sheet_name='Ind_2_Maria', index=False)
    df_rendActua_ERecono_indicador_2_Ordenado_Repelon_pivote.to_excel(writer, sheet_name='Ind_2_Repelon', index=False)
    # df_rendActua_ERecono_indicador_3_pivote.to_excel(writer, sheet_name='Indicador_3', index=False)
    df_indicador_4_maria_std_pivote.to_excel(writer, sheet_name='Indicador_4_MariaLaBaja', index=False)
    df_radicado_npn.to_excel(writer, sheet_name='Radicado_X_NPN', index=False)
    df_npn_croquis_estandarizado.to_excel(writer, sheet_name='Reporte_NPN_Croquis', index=False)

print(f"Se exporta XLSX {NOMBRE_XLSX}, en la ruta {DIRECTORIO_SALIDA_XLSX}")


Se exporta XLSX 20240617_RendimientoEquipoReconocimiento_BCGS.xlsx, en la ruta C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_2_Rendimientos_Reconocimiento
